In [1]:
import os
import sys
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
sys.path.append("../python")

import zillow_parse as zp

In [2]:
zillow_pages = zp.getZillowPages()

In [3]:
#add contents of urls to soup variable from each url
soup = BeautifulSoup(zillow_pages['req1'].content, 'html.parser')
soup1 = BeautifulSoup(zillow_pages['req2'].content, 'html.parser')

In [4]:
all_homes_df = pd.DataFrame()

In [5]:
all_homes_df = zp.gatherZillowDetails(all_homes_df, soup)
all_homes_df = zp.gatherZillowDetails(all_homes_df, soup1)

In [8]:
parsed_df = zp.parseZillowDetails(all_homes_df)

In [41]:
link = parsed_df.links[10]
print(link)

https://www.zillow.com/homedetails/4930-N-Damen-Ave-1-Chicago-IL-60625/2077702013_zpid/


'https://www.zillow.com/homedetails/2035-W-Leland-Ave-Chicago-IL-60625/2082999865_zpid/'

In [42]:
ZILLOW_FEATURES = ['basement', 'flooring', 'heating features', 'cooling features', 'laundry features', 
                   'total spaces', 'parking features', 'garage spaces', 'covered spaces', 'attached garage',
                   'construction materials', 'year built', 'hoa fee', 'annual tax amount']

In [43]:
r = s.get(link, headers=zp.REQ_HEADERS)

In [44]:
soup = BeautifulSoup(r.content, 'html.parser')

In [45]:
for span in soup.find_all('span'):
    span_txt = span.get_text()
    if ':' in span_txt:
        print(span_txt.split(':'))

['Zestimate®', '\xa0$852,067']
['Zestimate®', '\xa0$852,067']
['Est. payment', '\xa0']
['Zestimate®', '\xa0$852,067']
['Zestimate®', '\xa0$852,067']
['Est. payment', '\xa0']
['Type', '']
['Year built', '']
['Heating', '']
['Cooling', '']
['Parking', '']
['HOA', '']
['Price/sqft', '']
['Bedrooms', ' 4']
['Bathrooms', ' 4']
['Full bathrooms', ' 3']
['1/2 bathrooms', ' 1']
['Basement', ' None']
['Heating features', ' Natural Gas, Forced Air']
['Cooling features', ' Central Air']
['Appliances included', ' Range, Microwave, Dishwasher, Refrigerator, High End Refrigerator, Washer, Dryer, Stainless Steel Appliance(s), Wine Refrigerator']
['Total interior livable area', ' 3,000 sqft']
['Total number of fireplaces', ' 1']
['Fireplace features', ' Living Room']
['Total spaces', ' 1']
['Parking features', ' ']
['Garage spaces', ' 1']
['Covered spaces', ' 1']
['Other parking information', ' Garage(On Site, Detached)']
['Parcel number', ' 14073170300000']
['Special conditions', ' None']
['Home type

In [ ]:
#calculate the zestimate and insert into a dataframe
zillow_zestimate = []
for link in df['links']:
    r = s.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    home_value = soup.select_one('h4:contains("Home value")')
    if not home_value:
        home_value = soup.select_one('.zestimate').text.split()[-1]
    else:
        home_value = home_value.find_next('p').get_text(strip=True)
    zillow_zestimate.append(home_value)

cols=['zestimate']
zestimate_result = pd.DataFrame(zillow_zestimate, columns=cols)
# zestimate_result

#convert zestimate column to float, and remove , and $
zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace('$','')
zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace('/mo','')
zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace(',','')

#covert rows with non zestimate to 0
def non_zestimate(zestimate_result):
    if len(zestimate_result['zestimate']) > 20:
        return '0'
    elif len(zestimate_result['zestimate']) < 5:
        return '0'
    else:
        return zestimate_result['zestimate']

zestimate_result['zestimate'] = zestimate_result.apply(non_zestimate,axis=1)

# zestimate_result

#concat zestimate dataframe and original df
df = pd.concat([df, zestimate_result], axis=1)
df['zestimate'] = df['zestimate'].astype('float')

#create best deal column and sort by best_deal
df ['best_deal'] = df['prices'] - df['zestimate']
df = df.sort_values(by='best_deal')

df